In [2]:
# Libraries
import pandas as pd
import random
import json
from datasets import load_dataset
import re
import math
import random
import spacy

In [9]:
dataset = load_dataset("gsm8k", "main")
dataset["test"][5]

{'question': 'Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?',
 'answer': 'The discount price of one glass is 60/100 * 5 = $<<60/100*5=3>>3.\nIf every second glass is cheaper, that means Kylar is going to buy 16 / 2 = <<16/2=8>>8 cheaper glasses.\nSo for the cheaper glasses, Kylar is going to pay 8 * 3 = $<<8*3=24>>24.\nAnd for the regular-priced glasses, Kylar will pay 8 * 5 = $<<8*5=40>>40.\nSo in total Kylar needs to pay 24 + 40 = $<<24+40=64>>64 for the glasses he wants to buy.\n#### 64'}

In [ ]:
model_name = "TheBloke/Llama-2-7B-Chat-GGUF"
model_basename = "./llama-2-7b-chat.Q5_K_M.gguf"

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openchat/openchat_3.5")
model = AutoModelForCausalLM.from_pretrained("openchat/openchat_3.5")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
max_token = 500

In [3]:
def get_random_test_object(dataset):
    if "test" in dataset and len(dataset["test"]) > 0:
        random_index = random.randint(0, len(dataset["test"]) - 1)
        test_object = dataset["test"][random_index]
        
        # Assuming each object in the dataset has 'question' and 'answer' keys
        question = test_object.get("question", "No question found")
        answer = test_object.get("answer", "No answer found")
        
        return question, answer
    else:
        return None, None

In [4]:
def get_random_mutation(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path, header=None, encoding='utf-8', delimiter='.') 
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file_path, header=None, encoding='ISO-8859-1', delimiter='.') 

    random_prompt = random.choice(df[1].tolist())
    return random_prompt


In [5]:
def get_random_mutation_txt(txt_file_path):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove any leading/trailing whitespace and filter out empty lines
    prompts = [line.strip() for line in lines if line.strip()]
    
    if prompts:
        return random.choice(prompts)
    else:
        return "No mutation prompts found."

In [6]:
# write the task description here:
task_description = "Generate an instruction on how to solve the problem, based on the given question "

In [7]:
task_description1 = "Generate an instruction, or advice on how to solve a problem"

#### Hide

In [10]:
# getting mutated prompts
question, answer = get_random_test_object(dataset)

In [11]:
print(question)

In one hour, Ezra read twice as many books as Ahmed. Ezra has read 300 books this hour and decided to read 150 more. How many books have they read altogether?


In [12]:
print(answer)

If Ezra has read 300 books this hour and decided to read 150 more in the next hour, he has read a total of 300+150=<<300+150=450>>450
Since Ezra reads twice as many books as Ahmed, Ahmed has read 450/2=<<450/2=225>>225 books.
Together, Ahmed and Ezra has read 225+450=<<225+450=675>>675 books
#### 675


## Instruction Generation

In [13]:
def check_answer(answer, output):
    if not (isinstance(answer, str) and isinstance(output, str)):
        raise TypeError("Must be of type str")
    if re.search("\s" + answer +"\s*", output):
        # print("answer is correct")
        return 1
    # print("answer is wrong")
    return 0

In [14]:
def get_logic(answer):
    return re.findall("<<(.*?)>>", answer)

In [15]:
def tokenise_logic(logic):
    tokenised_logic = []
    operators = "+-/*="
    current_token = ""
    
    for c in logic:
        if c in operators:
            tokenised_logic.append(current_token)
            tokenised_logic.append(c)
            current_token = ""
        elif c == '.' or c.isnumeric():
            current_token += c
    tokenised_logic.append(current_token)
    
    return tokenised_logic

In [16]:
def check_logic_sentence(line, logic_tokens):
    i = 0

    for token in logic_tokens:
        if token in line:
            i += 1

    if len(logic_tokens) == i:
        return True
    return False

In [17]:
mutation_prompt = get_random_mutation_txt("mutation_prompts.txt")

In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM

class llamathwiz:
    def __init__(self):
        # self.tokenizer = AutoTokenizer.from_pretrained("meta-math/MetaMath-Llemma-7B")
        # self.model = AutoModelForCausalLM.from_pretrained("meta-math/MetaMath-Llemma-7B")
        self.tokenizer = tokenizer
        self.model = model
        # self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
        # self.model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
        self.gen = 0
        # self.llm = Llama(
        #     model_path=model_path,
        #     n_threads=2,
        #     n_batch=512,
        #     n_gpu_layers=32
        #     )

        
    def generate_instructions(self, question, task_description, max_tokens=100):
        formatted_input = f"{task_description} {question}"
        output = self.llm(formatted_input)

        return output["choices"][0]["text"]
    
    def generate_instructions1(self, num_instructions, max_tokens=1000, temperature=1.5):
        instructions = []
        for num in range(num_instructions):
            formatted_input = "Generate an instruction on how to solve a simple math problem without the need of a question"
            #added by leo
            input_ids = tokenizer(formatted_input, return_tensors="pt").input_ids
            outputs = model.generate(input_ids,
                                     max_new_tokens=int(max_tokens),
                                     temperature=temperature)
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(generated_text)
            # generated_text = self.llm(formatted_input)["choices"][0]["text"]
            instructions.append(generated_text)

        return instructions

#         formatted_input = f"Generate {num_instructions} separate instructions on how to solve a simple math problem without the need of a question."
#         generated_text = self.llm(formatted_input, max_tokens=2048)

#         return generated_text

    # def process_with_llm(self, prompt):
    #     return self.llm(prompt)["choices"][0]["text"]
    
    #added by leo
    def process_with_llm(prompt, model, tokenizer):
        # Tokenize input with attention mask
        inputs = tokenizer(prompt, return_tensors='pt', padding='max_length', max_length=512, truncation=True)
        
        # Extract input_ids and attention mask
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        # Generate a response using the model, considering attention mask
        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=1000)

        # Decode the generated tokens to get the output text
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return response
    
    def apply_mutation(self, instruction, mutation_prompt):
        input_ids = tokenizer.encode(f'{instruction} \n Rewrite this and apply the following mutation. Mutation: {mutation_prompt}', return_tensors='pt')
        output = model.generate(input_ids, max_length=1000)
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        # Example mutation - this can be customized based on your mutation logic
        return response
    
    def fitness(self, database_answer, output):
        final_answer = database_answer.split()[-1]

        score = 0

        logic = get_logic(database_answer)

        tokenised_logic_sentences = []

        for l in logic:
            tokenised_logic_sentences.append(tokenise_logic(l))

        total_score = 1 + len(tokenised_logic_sentences)

        for line in output.split("."):
            for sentence in tokenised_logic_sentences:
                if check_logic_sentence(line, sentence):
                    tokenised_logic_sentences.pop(tokenised_logic_sentences.index(sentence))
                    score += 1
                    break
        last_line = line

        num_answer = 'a'

        for word in last_line.split(" "):
            try:
                num_answer = str(int(word))
            except:
                pass

        score += check_answer(final_answer, num_answer)
        return score/total_score
    
    def iteration_first(self, num_instructions=1):
        self.gen = 1
        self.generated_prompts = []
        self.generated_answers = []
        self.iteration_questions = []
        self.scores = []
        
        for _ in range(num_instructions):
            question, database_answer = get_random_test_object(dataset)  # Fetch a random question from your dataset
            self.iteration_questions.append((question, database_answer))
            instruction = self.generate_instructions1(1)
            mutated_instruction = self.apply_mutation(instruction, mutation_prompt)
            self.generated_prompts.append(mutated_instruction)
            # self.generated_prompts.append(instruction)
            processed_output = self.process_with_llm(f'Q:{question} I:{mutated_instruction} A:') 
            self.generated_answers.append(processed_output)
            self.scores.append(self.fitness(database_answer, processed_output))
            
    
    def iteration_execute(self):
        self.gen += 1
        self.generated_prompts = []
        self.generated_answers = []
        self.iteration_questions = []
        self.scores = []

        mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
        for instruction in self.best_instructions:
            question, database_answer = get_random_test_object(dataset)  # Fetch a random question from your dataset
            self.iteration_questions.append((question, database_answer))
            mutated_instruction = self.apply_mutation(instruction, mutation_prompt)
            self.generated_prompts.append(mutated_instruction)
            processed_output = self.process_with_llm(f'Q:{question} I:{mutated_instruction} A:')
            self.generated_answers.append(processed_output)
            self.scores.append(self.fitness(database_answer, processed_output))
    
    def iteration_prepare(self):
        self.best_instructions = []
        self.find_best_scores(self.scores)
        self.best_instructions.append(self.replace_pos(self.best_instructions[0], self.best_instructions[1], ['ADV', 'ADJ', 'NOUN']))
        self.best_instructions.append(self.replace_pos(self.best_instructions[1], self.best_instructions[0], ['ADV', 'ADJ', 'NOUN']))
        for instr in self.generate_instructions1(4):
            self.best_instructions.append(instr)
        for i in range(2):
            mutation_prompt = get_random_mutation_txt("./Prompt-Engineering-OpenDI/mutation_prompts.txt")
            self.best_instructions.append(self.apply_mutation(self.best_instructions[i], mutation_prompt))
        
    def find_best_scores(self, scores):
        for i in range(2):
            maximum_val = 0
            maximum_index = 0
            while i < len(scores):
                if scores[i] > maximum_val:
                    maximum_val = scores[i]
                    maximum_index = i
                i += 1
            scores.pop(maximum_index)
            print(self.generated_prompts[maximum_index])
            self.best_instructions.append(self.generated_prompts.pop(maximum_index))
            
    def replace_pos(self, string1, string2, POS):

        """
        Replace tokens of specific parts of speech (POS) in string1 with corresponding
        tokens of the same POS from string2.

        Parameters:
        - string1 (str): The input string where certain POS will be replaced.
        - string2 (str): The reference string from which POS replacements will be taken.
        - POS (list): A list of POS tags to identify which tokens to replace in string1.

        Returns:
        - str: The modified string with replacements of tokens based on specified POS tags.

        Possible POS:
        "ADJ": "adjective",
        "ADP": "adposition",
        "ADV": "adverb",
        "AUX": "auxiliary",
        "CONJ": "conjunction",
        "CCONJ": "coordinating conjunction",
        "DET": "determiner",
        "INTJ": "interjection",
        "NOUN": "noun",
        "NUM": "numeral",
        "PART": "particle",
        "PRON": "pronoun",
        "PROPN": "proper noun",
        "PUNCT": "punctuation",
        "SCONJ": "subordinating conjunction",
        "SYM": "symbol",
        "VERB": "verb".
        """
        nlp = spacy.load("en_core_web_sm")
        doc1 = nlp(string1)
        doc2 = nlp(string2)

        new_tokens = []

        for token1 in doc1:
            if token1.pos_ in POS:
                #generating a list of all matching tokens
                matching_tokens = [token2.text for token2 in doc2 if token2.pos_ == token1.pos_]
                #use a random token if possible, or else use the same
                if matching_tokens:
                    new_token = random.choice(matching_tokens)
                else:
                    new_token = token1.text
            else:
                new_token = token1.text
            new_tokens.append(new_token)

        # Join the modified tokens to form the final string
        result = ' '.join(new_tokens)
        return result
    
    

In [22]:
ai = llamathwiz()

In [23]:
ai.iteration_first()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


KeyboardInterrupt: 

In [ ]:
# self.generated_prompts = []
# self.generated_answers = []
# self.iteration_questions = []
# self.scores = []

ai.scores

In [ ]:
ai.generated_answers[1]

In [ ]:
ai.iteration_questions[1][1]

In [ ]:
ai.iteration_prepare()

In [ ]:
len(ai.best_instructions)

In [ ]:
ai.iteration_execute()

In [ ]:
ai.generated_prompts[4]

In [ ]:
ai.iteration_first()
for i in range(49):
    ai.iteration_prepare()
    ai.iteration_execute()
    
ai.score()

In [ ]:
instructions = ai.generate_instructions1(3)

In [ ]:
print(instruction)

In [ ]:
for instr in instructions:
    print(instr + '\n')

In [ ]:
print(instructions)

In [ ]:
instructions["choices"][0]["text"]

In [ ]:
def fitness(database_answer, output):
    final_answer = database_answer.split()[-1]

    score = 0

    logic = get_logic(database_answer)

    tokenised_logic_sentences = []

    for l in logic:
        tokenised_logic_sentences.append(tokenise_logic(l))
    print(tokenised_logic_sentences)

    total_score = 1 + len(tokenised_logic_sentences)

    for line in output.split("."):
        for sentence in tokenised_logic_sentences:
            if check_logic_sentence(line, sentence):
                tokenised_logic_sentences.pop(tokenised_logic_sentences.index(sentence))
                score += 1
                break
    last_line = line
    print(tokenised_logic_sentences)

    num_answer = 'a'

    for word in last_line.split(" "):
        try:
            num_answer = str(int(word))
        except:
            pass

    score += check_answer(final_answer, num_answer)
    return score/total_score

In [ ]:
fitness('The $100 was divided into 3 + 2 = <<3+2=5>>5 parts.\nSo each part is $100/5 = $<<100/5=20>>20.\nSo Gerald received $20 x 3 = $<<20*3=60>>60.\nGerald bought a book at $<<10=10>>10.\nTherefore, Gerald was left with $60 - $10 = $<<60-10=50>>50.\n#### 50',
"2 + 3 = 5. Easy peasy, right? Now let's see how many apps Travis has on his tablet now:\nA:Travis has 61 - 9 = 52 apps on his tablet after deleting 9 apps he didn't use anymore. Then, he downloaded 18 more apps, so he has 52 + 18 = 70 apps on his tablet now!")